In [1]:
import os
import nd2
import napari
import pickle
import numpy as np
import pandas as pd
from skimage.io import imread
from skimage.measure import regionprops_table, profile_line
from cardiomyocytes_helper_functions import segment_actin_3D,calculate_orientation,find_fibers_orientation_v2,orientations_from_vertices, signal_from_vertices

In [3]:
path_dir = r'D:\data_analysis\2022_Sahana\data\Collagen\60x images'
path_vertices = r'D:\data_analysis\2022_Sahana\masks'

path_results = r'D:\data_analysis\2022_Sahana\results'
#path_df = os.path.join(path_results,im_name.replace('.nd2','_df.pkl'))

# order of channels
    # actin
    # paxilin
    # plakoglobin
    # DAPI


## Loop over the images

In [4]:
file_list = os.listdir(path_dir)

In [5]:
file_list

['092622_ring_PDMSAp_10MCol_647-Act_561-Pax_488-Plak_DAPI_001.nd2',
 '092622_ring_PDMSAp_10MCol_647-Act_561-Pax_488-Plak_DAPI_002.nd2',
 '092622_ring_PDMSAp_10MCol_647-Act_561-Pax_488-Plak_DAPI_003.nd2',
 '092622_ring_PDMSAp_10MCol_647-Act_561-Pax_488-Plak_DAPI_004.nd2',
 '092622_ring_PDMSAp_10MCol_647-Act_561-Pax_488-Plak_DAPI_005.nd2',
 '092622_ring_PDMSAp_10MCol_647-Act_561-Pax_488-Plak_DAPI_006.nd2',
 '092622_ring_PDMSAp_10MCol_647-Act_561-Pax_488-Plak_DAPI_007.nd2',
 '092622_ring_PDMSAp_10MCol_647-Act_561-Pax_488-Plak_DAPI_008.nd2',
 '092622_ring_PDMSAp_10MCol_647-Act_561-Pax_488-Plak_DAPI_009.nd2',
 '092622_ring_PDMSAp_10MCol_647-Act_561-Pax_488-Plak_DAPI_010.nd2',
 '092622_ring_PDMSAp_10MCol_647-Act_561-Pax_488-Plak_DAPI_011.nd2',
 '10142022_PDMSAp_ColIMed10_647-Act_561-Vinc_488-Jup_DAPI_001.nd2',
 '10142022_PDMSAp_ColIMed10_647-Act_561-Vinc_488-Jup_DAPI_002.nd2',
 '10142022_PDMSAp_ColIMed10_647-Act_561-Vinc_488-Jup_DAPI_003.nd2',
 '10142022_PDMSAp_ColIMed10_647-Act_561-Vinc_488

In [11]:
df_list = []
properties = ['label','area','centroid','bbox','eccentricity','orientation','intensity_mean','intensity_image','image']


for im_name in file_list:

    path_im = os.path.join(path_dir,im_name)

    # read in image
    path_im = os.path.join(path_dir,im_name)
    im = nd2.imread(path_im)

    # read in masks
    mask_path = os.path.join(path_vertices,im_name.replace('.nd2','_mask.png'))
    if os.path.isfile(mask_path):
        
        # open mask
        mask = imread(mask_path)

        # get actin channel
        image_actin = im[:,0,:,:]

        # segment actin volume
        image_actin_mask = segment_actin_3D(image_actin)

        # flatten segmented actin
        image_actin_mask_2D = np.max(image_actin_mask,axis=0)
    
        
        # calculate df
        im_flat_all_channels = np.max(im,axis=0)
        im_flat_all = np.append(im_flat_all_channels,np.expand_dims(image_actin_mask_2D,axis=0),axis=0)
        im_flat_all = np.moveaxis(im_flat_all,0,2)

        cell_measure = regionprops_table(mask, intensity_image = im_flat_all, properties = properties)
        df = pd.DataFrame(cell_measure)
        df['image_name'] = im_name

        ''''
        # save individual df
        path_df = os.path.join(path_results,im_name.replace('.nd2','_df.pkl'))
        df.to_pickle(path_df)
        '''

        # collect df
        df_list.append(df)
          
    
    else:
        print(f'{im_name} does not have a mask')



intensity normalization: min-max normalization with NO absoluteintensity upper bound
intensity normalization: min-max normalization with NO absoluteintensity upper bound
092622_ring_PDMSAp_10MCol_647-Act_561-Pax_488-Plak_DAPI_003.nd2 does not have a mask
092622_ring_PDMSAp_10MCol_647-Act_561-Pax_488-Plak_DAPI_004.nd2 does not have a mask
092622_ring_PDMSAp_10MCol_647-Act_561-Pax_488-Plak_DAPI_005.nd2 does not have a mask
092622_ring_PDMSAp_10MCol_647-Act_561-Pax_488-Plak_DAPI_006.nd2 does not have a mask
092622_ring_PDMSAp_10MCol_647-Act_561-Pax_488-Plak_DAPI_007.nd2 does not have a mask
092622_ring_PDMSAp_10MCol_647-Act_561-Pax_488-Plak_DAPI_008.nd2 does not have a mask
092622_ring_PDMSAp_10MCol_647-Act_561-Pax_488-Plak_DAPI_009.nd2 does not have a mask
092622_ring_PDMSAp_10MCol_647-Act_561-Pax_488-Plak_DAPI_010.nd2 does not have a mask
092622_ring_PDMSAp_10MCol_647-Act_561-Pax_488-Plak_DAPI_011.nd2 does not have a mask
10142022_PDMSAp_ColIMed10_647-Act_561-Vinc_488-Jup_DAPI_001.nd2 d

In [17]:
# concatenate all data frames
df_all = pd.concat(df_list,ignore_index=True)

In [18]:
df_all

,label,area,centroid-0,centroid-1,bbox-0,bbox-1,bbox-2,bbox-3,eccentricity,orientation,intensity_mean-0,intensity_mean-1,intensity_mean-2,intensity_mean-3,intensity_mean-4,intensity_image,image,image_name
0,1,21500,416.317721,305.246605,351,191,512,448,0.927055,-1.159706,1662.444326,627.910605,459.778279,210.450419,0.403860,"[[[0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, ...","[[False, False, False, False, False, False, Fa...",092622_ring_PDMSAp_10MCol_647-Act_561-Pax_488-...
1,2,17456,326.136801,245.599736,269,135,391,353,0.863103,-1.429548,1461.129927,614.861595,409.339253,204.224278,0.417106,"[[[0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, ...","[[False, False, False, False, False, False, Fa...",092622_ring_PDMSAp_10MCol_647-Act_561-Pax_488-...
2,3,6480,219.750309,130.168056,148,98,279,171,0.860131,0.197693,1872.356173,696.766204,245.666204,283.347222,0.409722,"[[[0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, ...","[[False, False, False, False, False, False, Fa...",092622_ring_PDMSAp_10MCol_647-Act_561-Pax_488-...
3,4,17279,221.896348,231.103131,141,144,291,332,0.792619,1.247677,1604.532785,542.639215,447.338909,199.819434,0.458070,"[[[0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, ...","[[False, False, False, False, False, False, Fa...",092622_ring_PDMSAp_10MCol_647-Act_561-Pax_488-...
4,1,14349,251.898042,353.204544,160,252,334,447,0.747291,1.124676,1638.013729,492.773922,531.278626,170.766465,0.422956,"[[[0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, ...","[[False, False, False, False, False, False, Fa...",092622_ring_PDMSAp_10MCol_647-Act_561-Pax_488-...


In [16]:
df_all.image_name.tolist()

['092622_ring_PDMSAp_10MCol_647-Act_561-Pax_488-Plak_DAPI_001.nd2',
 '092622_ring_PDMSAp_10MCol_647-Act_561-Pax_488-Plak_DAPI_001.nd2',
 '092622_ring_PDMSAp_10MCol_647-Act_561-Pax_488-Plak_DAPI_001.nd2',
 '092622_ring_PDMSAp_10MCol_647-Act_561-Pax_488-Plak_DAPI_001.nd2',
 '092622_ring_PDMSAp_10MCol_647-Act_561-Pax_488-Plak_DAPI_002.nd2']

In [ ]:


# save concatenated data frames together
df_all.to_pickle(os.path.join(path_results,'df_all.pkl'))